In [0]:
!pip install transformers

     |████████████████████████████████| 491kB 18.4MB/s 
     |████████████████████████████████| 1.0MB 53.1MB/s 
     |████████████████████████████████| 870kB 53.9MB/s 
     |████████████████████████████████| 3.8MB 60.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=f73ffd2d8b185159a15ac7499695d4c2e487b7ae6b93013f422ad9596758c4c2
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
%tensorflow_version 2.x
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset

import os
import math
import time

from transformers import BertModel, BertForMaskedLM, BertConfig, BertPreTrainedModel
import numpy as np

TensorFlow 2.x selected.


In [0]:
import sys
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/projet-alexandrin/'  # your new root path

sys.path.append(os.path.join(root_path, 'notebooks')) # for importing from utils.py

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
test = [txt for i, txt in enumerate(token_to_index)]

In [0]:
#data import
with open(root_path+'data/corneille_racine_mm.txt', encoding = 'utf-8') as file:
    data = file.read().lower()

#Tokenization

sequences = data.split('\n')
sequences = list(filter(lambda x: len(x)>5,sequences))

#data import
# sequences = []
# with open(root_path+'data/corneille_racine_mm.txt', encoding = 'utf-8') as fin:
#     for line in fin:
#       sequences.append(line)

all_tokens=['[CLS]', '[SEP]', '[MASK]']
for seq in sequences:
    for token in seq.split(' '):
        all_tokens+=[token]
token_to_index = dict(
    [(txt, int(i)) for i, txt in enumerate(all_tokens)])
token_to_index = dict(
    [(txt, int(i)) for i, txt in enumerate(token_to_index)])
index_to_token = dict(
    (int(i), txt) for txt, i in token_to_index.items())

vocab_size = len(token_to_index)
print('vocab_size ',vocab_size)

input_tokens = []
output_tokens = []
expected_tokens = []
special_range = list(range(12))+[23]+list(range(12, 23))

for i in range(0,len(sequences)-4,2):
    for j in special_range:
      # on prend les vers deux à deux
      input_1 = ('[CLS] '+sequences[i]+' [SEP]').split(' ')
      input_2 = (sequences[i+1]+' [SEP]').split(' ')
      expected_output_1 = (sequences[i+2]+' [SEP]').split(' ')
      expected_output_2 = (sequences[i+3]+' [SEP]').split(' ')
      # the model will fill the input_3 and input_4 mask per mask starting from the left to the right

      input_3 = ['[MASK]','[MASK]','[MASK]','[MASK]','[MASK]','[MASK]',
                '[MASK]','[MASK]','[MASK]','[MASK]','[MASK]','[MASK]','[SEP]']
      input_4 = ['[MASK]','[MASK]','[MASK]','[MASK]','[MASK]','[MASK]',
                '[MASK]','[MASK]','[MASK]','[MASK]','[MASK]', expected_output_2[-2],'[SEP]']  # putting expected_output_1 to mimic real conditions
      if j<12 or j==23:
          input_3[:j] = expected_output_1[:j]
          input_4[-2] = '[MASK]'
      elif j>=12:
          input_3 = expected_output_1
          input_4[:j-12] = expected_output_2[:j-12]

      if len(input_1) == 14 and len(input_2) == 13 and len(expected_output_1) == 13 and len(expected_output_2) == 13:
        if j<12:
            input_tokens.append(input_1+input_2+input_3+input_4)
            output_tokens.append([expected_output_1[j]])
        else:
            input_tokens.append(input_1+input_2+input_3+input_4)
            output_tokens.append([expected_output_2[j-12]])


num_input = len(input_tokens)
print('num_input ',num_input)

max_seq_len = len(input_tokens[0])
print('max_seq_len ',max_seq_len)

input_ids = []
output_ids = []

for i in range(num_input):
  tmp_input = []
  for j in range(max_seq_len):
      try:
        tmp_input.append(token_to_index[input_tokens[i][j]])
      except:
        pass
        # print(i,j)
  input_ids.append(tmp_input)
  output_ids.append(token_to_index[output_tokens[i][0]])

print('Sequences')
print(sequences[0:2])
print('Tokens')
print(input_tokens[0])
print('Index')
print(input_ids[0])

#Attention Mask
print('Attention MASK')
attn_mask = []

for token in input_tokens:
    attn_mask.append([1 if tk != '[MASK]' else 0 for tk in token])
print(attn_mask[0])

#Segment Tokens
print('Segment Tokens')
seg_ids = []
for token in input_tokens:
    seg_ids.append([0 if i < 27 else 1 for i,tk in enumerate(token)])
print(seg_ids[0])

vocab_size  3201
num_input  949104
max_seq_len  53
Sequences
['j@ t@ la vou a mi mon mal ai ltin ku rabl', 'j@ ni sai kun r@ maid e jan sywi lzin ka pabl']
Tokens
['[CLS]', 'j@', 't@', 'la', 'vou', 'a', 'mi', 'mon', 'mal', 'ai', 'ltin', 'ku', 'rabl', '[SEP]', 'j@', 'ni', 'sai', 'kun', 'r@', 'maid', 'e', 'jan', 'sywi', 'lzin', 'ka', 'pabl', '[SEP]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
Index
[0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1, 3, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]
Attention MASK
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Segment Tokens
[0, 0, 0, 0, 0, 0, 

In [0]:
for i in range(4):
  print(sequences[i])

j@ t@ la vou a mi mon mal ai ltin ku rabl
j@ ni sai kun r@ maid e jan sywi lzin ka pabl
l@ shan j@@ s@ rai just a prai tan d@ ri goer
mai mal gre se de din me lit aa tou mon koer


In [0]:
input_ids = np.array(input_ids)
attn_mask = np.array(attn_mask)
seg_ids = np.array(seg_ids)
output_ids = np.array(output_ids)

In [0]:
num_layers = 4
num_heads = 4

config = BertConfig(vocab_size=3201,
        hidden_size=768,
        num_hidden_layers=num_layers,
        num_attention_heads=num_heads,
        intermediate_size=3072,
        hidden_act="gelu",
        hidden_dropout_prob=0.1,
        attention_probs_dropout_prob=0.1,
        max_position_embeddings=512,
        type_vocab_size=2,
        initializer_range=0.02,
        layer_norm_eps=1e-12)

In [0]:
# Copy from https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317

def top_k_top_p_filtering(logits, top_k=0, top_p=0.9, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k >0: keep only top k tokens with highest probability (top-k filtering).
            top_p >0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(torch.nn.functional.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

In [0]:
# Copy from https://github.com/huggingface/transformers/blob/master/src/transformers/modeling_bert.py

def gelu(x):
    """ Original Implementation of the gelu activation function in Google Bert repo when initially created.
        For information: OpenAI GPT's gelu is slightly different (and gives slightly different results):
        0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
        Also see https://arxiv.org/abs/1606.08415
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

class BertPredictionHeadTransform(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        if isinstance(config.hidden_act, str):
            self.transform_act_fn = gelu
        else:
            self.transform_act_fn = config.hidden_act
        self.LayerNorm = torch.nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.transform_act_fn(hidden_states)
        hidden_states = self.LayerNorm(hidden_states)
        return hidden_states


class BertLMPredictionHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.transform = BertPredictionHeadTransform(config)

        # The output weights are the same as the input embeddings, but there is
        # an output-only bias for each token.
        self.decoder = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        self.bias = nn.Parameter(torch.zeros(config.vocab_size))

        # Need a link between the two variables so that the bias is correctly resized with `resize_token_embeddings`
        self.decoder.bias = self.bias

    def forward(self, hidden_states):
        hidden_states = self.transform(hidden_states)
        hidden_states = self.decoder(hidden_states) + self.bias
        return hidden_states

class BertOnlyMLMHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.predictions = BertLMPredictionHead(config)

    def forward(self, sequence_output):
        prediction_scores = self.predictions(sequence_output)
        return prediction_scores

In [0]:
class AlexendrinePredictor(BertPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        #Instantiating BERT model object 
        self.bert = BertModel(config)
        #Classification layer
        self.cls = BertOnlyMLMHead(config)

        self.init_weights()

    def forward(self, input_ids, attention_mask, token_type_ids):

        #Feeding the input to BERT model to obtain contextualized representations
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        sequence_output = outputs[0]
        prediction_scores = self.cls(sequence_output)

        outputs = (prediction_scores,)# + outputs[2:]

        return outputs

In [0]:
net = AlexendrinePredictor(config)
# net.from_pretrained(root_path + 'save' + str(num_layers) + '/')  # does not load weights apparently!
net.load_state_dict(torch.load(root_path + 'save' + str(num_layers) + '/pytorch_model.bin'))  # this seems to work
net = net.cuda()
criterion = nn.CrossEntropyLoss()

In [0]:
opti = optim.Adam(net.parameters(), lr = 1e-6)

In [0]:
EPOCH = 3
batch_size = 128
temperature = 0.7  # cf post https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317#gistcomment-2918768

for ep in range(EPOCH):
    L = [] # We are going to average each loss all n batchs.
    min_Loss = 10 # And Save the best average loss.
    
    permutation = torch.randperm(len(input_ids))      # Shuffle Data
    print('Epoch {}/{}'.format(ep+1,EPOCH))
    Max_ind = (len(input_ids)//batch_size)*batch_size # Avoid issues when len(input_ids) % batch_size != 0
    start = time.time()
    
    for it in range(0, Max_ind, batch_size):
        indices = permutation[it:it+batch_size]
        
        seq = input_ids[indices]
        at_m = attn_mask[indices]
        s_ids = seg_ids[indices]
        lab = output_ids[indices]

        first_mask_indices = np.argmax(seq == 2, axis = 1)  # first mask

        seq_tensor = torch.tensor(seq).to(torch.long).cuda()
        at_m_tensor = torch.tensor(at_m).cuda()
        s_ids_tensor = torch.tensor(s_ids).cuda()
        lab_tensor = torch.tensor(lab).cuda()
        
        #Clear gradients
        opti.zero_grad()  

        #Obtaining the logits from the model
        outputs = net(input_ids = seq_tensor,
                      attention_mask = at_m_tensor,
                      token_type_ids = s_ids_tensor)
        
        logits = torch.cat([torch.unsqueeze(outputs[0][i][first_mask_indices[i]], 0) for i in range(batch_size)], axis=0).cuda()

        # logits = torch.cat([torch.unsqueeze(top_k_top_p_filtering(outputs[0][i][first_mask_indices[i]] / temperature, filter_value=-10000), 0) for i in range(batch_size)], axis=0)

        #Computing loss
        loss = criterion(logits, lab_tensor.long())

        #Backpropagating the gradients
        loss.backward()
        L.append(loss.item())

        #Optimization step
        opti.step()

        if (it//batch_size + 1) % 100 == 0:
            print("  Iteration {}/{} done. Average loss: {:.4f}. Time: {} secs".format(it//batch_size+1, len(input_ids)//batch_size + 1, np.mean(L), int(time.time() - start)))
            start = time.time()
            L = []
            
        if (it//batch_size + 1) % 500 == 0:
            net.save_pretrained(root_path + 'save' + str(num_layers) + '/')
            print("Model saved.")
            # if np.mean(L)<min_Loss:
            #   net.save_pretrained(root_path + '/save' + str(num_layers) + '/')
            #   min_Loss = np.mean(L)
            #   L=[]

net.save_pretrained(root_path + 'save' + str(num_layers) + '/')
print("Model saved.")

Epoch 1/3
  Iteration 100/7415 done. Average loss: 4.0155. Time: 29 secs
  Iteration 200/7415 done. Average loss: 3.9949. Time: 29 secs
  Iteration 300/7415 done. Average loss: 3.9765. Time: 29 secs
  Iteration 400/7415 done. Average loss: 3.9971. Time: 29 secs
  Iteration 500/7415 done. Average loss: 4.0213. Time: 29 secs
Model saved.
  Iteration 600/7415 done. Average loss: 3.9986. Time: 29 secs
  Iteration 700/7415 done. Average loss: 3.9914. Time: 29 secs
  Iteration 800/7415 done. Average loss: 3.9763. Time: 29 secs
  Iteration 900/7415 done. Average loss: 4.0042. Time: 29 secs
  Iteration 1000/7415 done. Average loss: 3.9823. Time: 29 secs
Model saved.
  Iteration 1100/7415 done. Average loss: 4.0127. Time: 29 secs
  Iteration 1200/7415 done. Average loss: 4.0058. Time: 29 secs
  Iteration 1300/7415 done. Average loss: 3.9966. Time: 29 secs
  Iteration 1400/7415 done. Average loss: 4.0005. Time: 29 secs
  Iteration 1500/7415 done. Average loss: 4.0021. Time: 29 secs
Model saved.


In [0]:
def generate_verses(verse1, verse2, temperature = 0.7, top_k = 0, top_p = 0.9):
    """input should be strings in metrometre format"""
    
    input_tokens = []
    output_tokens = []
    expected_tokens = []
    
    special_range = list(range(12))+[23]+list(range(12, 23))

    input_1 = ('[CLS] '+ verse1 +' [SEP]').split(' ')
    input_2 = (verse2 +' [SEP]').split(' ')
    assert len(input_1) == 14 and len(input_2) == 13, "verses should be alexandrines"
    input_3 = ['[MASK]','[MASK]','[MASK]','[MASK]','[MASK]','[MASK]',
                '[MASK]','[MASK]','[MASK]','[MASK]','[MASK]','[MASK]','[SEP]']
    input_4 = ['[MASK]','[MASK]','[MASK]','[MASK]','[MASK]','[MASK]',
                '[MASK]','[MASK]','[MASK]','[MASK]','[MASK]','[MASK]', '[SEP]'] 
    
    input_tokens = input_1 + input_2 + input_3 + input_4

    input_ids = np.array([token_to_index[input_tokens[j]] for j in range(len(input_tokens))])
        
    seg_ids = [int(i >= 27)for i in range(max_seq_len)]
    
    for i in special_range:
        attn_mask = [1 if tk != 2 else 0 for tk in input_ids]

        seq_tensor = torch.tensor(input_ids).unsqueeze(0).cuda()
        at_m_tensor = torch.tensor(attn_mask).unsqueeze(0).cuda()
        s_ids_tensor = torch.tensor(seg_ids).unsqueeze(0).cuda()

        first_mask_idx = np.argmax(input_ids == 2)  # first mask

        outputs = net(input_ids = seq_tensor,
                      attention_mask = at_m_tensor,
                      token_type_ids = s_ids_tensor)

        if i != 23:
            # Keep only the last token predictions of the first batch item (batch size 1), apply a temperature coefficient and filter
            logits = outputs[0][0][first_mask_idx] / temperature
            filtered_logits = top_k_top_p_filtering(logits, top_k=top_k, top_p=top_p)

            # Sample from the filtered distribution
            probabilities = torch.nn.functional.softmax(filtered_logits, dim=-1)
            id_pred = torch.multinomial(probabilities, 1).item()
            input_ids[first_mask_idx] = id_pred
        else:
            id_pred = outputs[0][0][first_mask_idx].max(0)[1].item()  # we want the rhyme
            input_ids[-2] = id_pred
            
    
    verse3 = ' '.join([index_to_token[input_ids[j]] for j in range(27, 39)])
    verse4 = ' '.join([index_to_token[input_ids[j]] for j in range(40, 52)])
    
    return verse3, verse4

In [0]:
import transformer_fr_to_mm

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


INFO:absl:SubwordTextEncoder build: trying min_token_count 10541
INFO:absl:SubwordTextEncoder build: trying min_token_count 5270
INFO:absl:SubwordTextEncoder build: trying min_token_count 2635
INFO:absl:SubwordTextEncoder build: trying min_token_count 1317
INFO:absl:SubwordTextEncoder build: trying min_token_count 658
INFO:absl:SubwordTextEncoder build: trying min_token_count 329
INFO:absl:SubwordTextEncoder build: trying min_token_count 164
INFO:absl:SubwordTextEncoder build: trying min_token_count 82
INFO:absl:SubwordTextEncoder build: trying min_token_count 41
INFO:absl:SubwordTextEncoder build: trying min_token_count 20
INFO:absl:SubwordTextEncoder build: trying min_token_count 10
INFO:absl:SubwordTextEncoder build: trying min_token_count 15
INFO:absl:SubwordTextEncoder build: trying min_token_count 12
INFO:absl:SubwordTextEncoder build: trying min_token_count 11
INFO:absl:SubwordTextEncoder build: trying min_token_count 57271
INFO:absl:SubwordTextEncoder build: trying min_token_co

8038 4612
2to1: Latest checkpoint restored!!
tensorflow_to_mm: loaded everything


In [0]:
def translate(verse, top_p=0.9, temp=0.7, verbose=True):
  return transformer_fr_to_mm.translate(verse, top_p=top_p, temp=temp, verbose=verbose, direction_1to2=False)

In [0]:
temp = 0.7
top_k = 0
top_p = 0.9
temp2 = 0.6
top_k2 = 1
top_p2 = 0.9

for i in range(5):
  idx = np.random.randint(len(sequences)//2)
  print(idx)
  verse1 = sequences[2*idx]
  verse2 = sequences[2*idx+1]
  verse3, verse4 = generate_verses(verse1, verse2, temperature=temp, top_k=top_k, top_p=top_p)
  print("Input 1:  {}".format(verse1))
  print("Input 2:  {}".format(verse2))
  print("Output 1: {}".format(verse3))  
  print("Output 2: {}".format(verse4))
  print("Traduction Output 1: {}".format(translate(verse3, top_p=top_p2, temp=temp2, verbose=False)))
  print("Traduction Output 2: {}".format(translate(verse4, top_p=top_p2, temp=temp2, verbose=False)))

10903
Input 1:  j@ veu tan te pour tanlt an kor un oo tr@ vwa
Input 2:  a m@@ ne po li eukt e si j@ l@ ran vwa
Output 1: an fin j@ nai pwin just e d@ tou t@@ la vi
Output 2: a vaik d@ la r@ grai di ny@ d@ me r@ vi
Traduction Output 1: enfin je n ai point juste et de toute la vie
Traduction Output 2: avec de la regret digne de mes 
15631
Input 1:  e ki m@ port an kor d@ plulz e tran j@@ kou
Input 2:  vwa si sai san rai zon k@ jan lne tai ja lou
Output 1: e kan ltil s@ rai zonln a prai s@ gran sou pir
Output 2: si vo tr@ koer an kor a vai ra de kon dir
Traduction Output 1: et quand il se raison après ce grand soupir
Traduction Output 2: si votre coeur encore avait ravi des dire
12174
Input 1:  l@ myinln ai san pa raiy pywis k@ j@ vou lzan bras
Input 2:  jai ron pu voo dis kour da se moo vai z@@ graas
Output 1: sai ltin so ra m@@ jour lza van je la dou soer
Output 2: e ma ri te se lyeu d@ se d@ le lza soer
Traduction Output 1: c est jour à venger la douceur
Traduction Output 2: et ma témé

In [0]:
text = []
for i in range(5):
    verse3, verse4 = generate_verses(verse3, verse4, temperature=temp, top_k=top_k, top_p=top_p)
    print(verse3)
    print(verse4)
    text.append(translate(verse3, top_p=top_p2, temp=temp2, verbose=False))
    text.append(translate(verse4, top_p=top_p2, temp=temp2, verbose=False))
print()
for s in text:
    print(s)

e j@ vou lzaa sair ve d@ lywi di fe ni e
j@ n@ pywi vou re soudr a vaik un pair se e
mai d@ sai t@@ gran koer n@ vou de nye sur mwa
sai nyoer si vou l@ vwa pour vou a me de fwa
si vou n@ vou lzan re k@ vou pou ve re nye
e s@ rai vou de fan d@ kon di ny@@ de nye
vou pou ve vou par tir d@ plu di ny@ d@ graas
par de ja mai tou jour d@ ma pair dr@ l@ nas
par le j@ pywi vou soel d@ sa vwar mon lna mour
e me blou i no me n@ la peult a prai jour

et je vous a de lui félicité
je ne puis vous résoudre avec un père 
mais de cette grand coeur ne vous daigniez sur moi
seigneur si vous le voit pour vous à mes démfois
si vous ne vous en verrez que vous pouvez régner
et serait vous défend de digne dédaigner
vous pouvez vous partir de plus digne de grâce
pardonnez mais toujours de ma perdre le menace
 / je puis vous seul de savoir mon amour
et m ant ne la peut après jour


et qu il ait sa constance et l amour qui le cours  
on ne croit pas mon coeur avec ces contre jours  

vous ne régnez son pas assez faire défendre  
mais que la fin de même où la faire m entendre

que pour un roi ma soeur par vos intelligence  
si je n ai point de vous et dans une assurance

mais ce n est pas un coeur que l amour vous égale  
osez vous arrêter son hymen de rivale

et l attendant ses efforts ce coeur qui dans mes yeux  
qui n attentat rien contre leur confus furieux

mais ma gloire de gloire et cela de mes yeux  
et quand les ce moment ne serait pas des dieux

seigneur à ce discours me fait voir son visage  
avec elle est à vous pour régner son courage

et le dit si je porte en ses telles conquêtes  
et j ai vu le pouvoir que mes mains de sa tête

et ne me saurait point que la mort de sa mère  
mais pour me faire voir que je ne puis sans père

hé seigneur de ces lieux je verrai me prétendre  
et si je ne puis rien je ne me puis comprendre

et pour un grand courage il est encor ce jour  
pour vous voir avec moi de votre juste amour